# Docs generator for Image only transforms 

This notebook helps to create docs with examples of transforms.
Instruction:

1. Change the value of `augmentation_name` to the name of augmentation you want to create docs for.
2. Change the value of `options` list, each item of the list should correspond to the set of acceptable parameters for your transformation, that you want to demonstrate in docs.
3. Optional. Change the path to the image you want to use for the demonstration.
4. "Run all" the notebook.
5. Demonstration image and rst document will be created and altered automatically.
6. Commit, push, and make PR according to the contribution instruction.

In [ ]:
#change this variables
augmentation_name = 'RGBShift' 
options = ['(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1.0)',
           '(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1.0)',
           '(r_shift_limit=(20,21) , g_shift_limit=0, b_shift_limit=0, p=1.0)',
           '(r_shift_limit=0 , g_shift_limit=(-21,-20), b_shift_limit=0, p=1.0)',
           '(r_shift_limit=0, g_shift_limit=0, b_shift_limit=(30,31), p=1.0)',
          ]

In [ ]:
path_to_the_docs = '../docs/augs_overview/image_only/'
path_to_image = 'images/original_parrot.jpg'

In [ ]:
import albumentations as A
import numpy as np

import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
%matplotlib inline

In [ ]:
# reading the image
image = cv2.imread(path_to_image)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
# creating the demo image
num_options = len(options)

# grid shape depends on number of examples
if num_options == 1:
    f, ax = plt.subplots(1, 2, figsize=(15, 10))
elif num_options == 3:
    f, ax = plt.subplots(2, 2, figsize=(15, 10))
else:
    rows = (num_options +1) // 3 + ((num_options +1) % 3 != 0)
    f, ax = plt.subplots(rows, 3, figsize=(21, rows * 5))

# plotting the original image
ax = ax.flatten()
ax[0].imshow(image)
ax[0].set_title('1. Original image')
ax[0].axis('off')

# applying all provided variations of transformation and plotting the result
for i in range(num_options):
    exec('augmentation = A.' + augmentation_name + options[i])
    ax[i+1].imshow(augmentation(image = image)['image'])
    title = str(i+2) + '. ' + augmentation_name + options[i]
    if len(title) > 30:
        divider_pos = title.find(', ', 25)
        title = title[:divider_pos+1] + '\n' + title[divider_pos+1:]
    ax[i+1].set_title(title)
    ax[i+1].axis('off')

# hiding unused axis
for j in range(i+1, len(ax)):
    ax[j].axis('off')
    
f.savefig(os.path.join(path_to_the_docs, 'images', augmentation_name + ".jpg"), bbox_inches='tight')

In [ ]:
# opening the rst file
file = open(os.path.join(path_to_the_docs, 'image_only.rst'),"a")

# filling the header
file.writelines('\n'.join(
                [
                '',
                augmentation_name,
                '-------------------',
                '',
                '',
                'API link: :class:`~albumentations.augmentations.transforms.' + augmentation_name + '`',
                '',
                '',
                '1. Original image',
                ]))
file.writelines('\n')

# filling the list of the options
file.writelines('\n'.join(
                [str(i+2) + '. :code:`' + augmentation_name + options[i] +'`' for i in range(num_options)]))
file.writelines('\n')

# adding the image
file.writelines('\n'.join(
                [
                 '',
                 '.. figure:: ./images/' + augmentation_name + '.jpg',
                 '    :alt: ' + augmentation_name + ' image'
                ]))
file.writelines('\n')
file.close() 